# Valuation for Qlib Training

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

## Load Results

We first read all the results that qlib training genereted

In [ ]:
exp_name = 'onlypred_test/oos'

# read result
path = Path(f'../data/intermediate/results/{exp_name}')
pred = pd.read_parquet(path)
pred = pred.loc[~pred.index.duplicated(keep='last')]

# read return data and benchmark data
ret = pd.read_parquet('../data/intermediate/forward_return/1d_open_open.parquet')
benchmark = pd.read_parquet('../data/index/zz500.parquet', columns=['ret'])
benchmark = benchmark.loc[pred.index.levels[0]]

# calculate the top layer return
select = pred.groupby(level=0)['score'].apply(lambda x: x.sort_values(ascending=False).iloc[:int(len(x) // 10)]).droplevel(0).index
top_ret = ret.loc[select]

# initialize weights
weights = pd.Series(np.ones_like(select), index=select).groupby(level=0).apply(lambda x: x / x.sum())

# calculate the turnover by weights
turnover = (weights.unstack().fillna(0).shift(1) - weights.unstack().fillna(0)).fillna(0).stack()
turnover = turnover[turnover > 0].groupby(level=0).sum()
buy_turnover = turnover[turnover >= 0].groupby(level=0).sum()
buy_turnover.iloc[0] = 1

## Cumulative Return

In [ ]:
profit = top_ret.groupby(level=0)['label'].apply(lambda x: 
    (x * weights.loc[x.index]).sum() / weights.loc[x.index].sum()).shift(2).fillna(0)
profit = profit - 0.001 * buy_turnover
profit = pd.concat([profit, benchmark], axis=1)
profit.columns = ['profit', 'benchmark']
profit['excess'] = profit['profit'] - profit['benchmark']
(1 + profit).cumprod().plot();

## Max Drawdown

In [ ]:
mmd = (((1 + profit).cumprod() - (1 + profit).cumprod().cummax()) / (1 + profit).cumprod().cummax())
mmd.plot();

## Turnover

In [ ]:
buy_turnover.plot();

## IC

In [ ]:
pred_label = pd.concat([pred, ret], axis=1, join='inner')

In [ ]:
ic = pred_label.groupby(level=0).corr(method='spearman').loc[(slice(None), "label"), "score"].droplevel(1)
ic.rolling(10).mean().plot(title='ic rolling 10-day mean');